In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [38]:
result = {'date':[], '종가':[], '전일대비':[], '등락율':[]}

# for page in range(91,552, 1):
for page in range(91,555, 1):
  url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn'
  query = {
    'marketindexCd':'IRR_CALL',
    'page' : page
  }
  resp = requests.get(url, query)
  soup = BeautifulSoup(resp.content, 'lxml')
  date_tag = soup.select('table.tbl_exchange.today tbody tr td.date')
  num_tag = soup.select('table.tbl_exchange.today tbody tr td.num')
  
  for i in range(len(date_tag)):
    result['date'].append(date_tag[i].text.strip())
  
  for j in range(0, len(num_tag), 3):
    result['종가'].append(num_tag[j].text.strip())
  for j in range(1, len(num_tag), 3):
    result['전일대비'].append(num_tag[j].img['alt']+num_tag[j].text.strip())
  for j in range(2, len(num_tag), 3):
    result['등락율'].append(num_tag[j].text.strip())


In [39]:
df = pd.DataFrame(result)
df 

,date,종가,전일대비,등락율
0,2018.01.02,1.52,하락0.06,-3.80%
1,2017.12.29,1.58,상승0.03,+ 1.94%
2,2017.12.28,1.55,상승0.04,+ 2.65%
3,2017.12.27,1.51,하락0.01,-0.66%
4,2017.12.26,1.52,상승0.01,+ 0.66%
...,...,...,...,...
3243,2004.12.14,3.28,보합0.00,0.00%
3244,2004.12.13,3.28,보합0.00,0.00%
3245,2004.12.10,3.28,상승0.01,+ 0.31%
3246,2004.12.09,3.27,보합0.00,0.00%


In [167]:
df = pd.DataFrame(result, columns=result.keys())

df.to_csv('call_rate.csv', index  = False)

In [168]:
call_rate = df
call_rate

,date,종가,전일대비,등락율
0,2018.01.02,1.52,하락0.06,-3.80%
1,2017.12.29,1.58,상승0.03,+ 1.94%
2,2017.12.28,1.55,상승0.04,+ 2.65%
3,2017.12.27,1.51,하락0.01,-0.66%
4,2017.12.26,1.52,상승0.01,+ 0.66%
...,...,...,...,...
3243,2004.12.14,3.28,보합0.00,0.00%
3244,2004.12.13,3.28,보합0.00,0.00%
3245,2004.12.10,3.28,상승0.01,+ 0.31%
3246,2004.12.09,3.27,보합0.00,0.00%


In [267]:
call_rate = pd.read_csv('./call_rate.csv')

In [268]:
call_rate.tail()

,date,종가,전일대비,등락율
3243,2004.12.14,3.28,보합0.00,0.00%
3244,2004.12.13,3.28,보합0.00,0.00%
3245,2004.12.10,3.28,상승0.01,+ 0.31%
3246,2004.12.09,3.27,보합0.00,0.00%
3247,2004.12.08,3.27,상승0.08,+ 2.51%


In [269]:
import datetime as dt
call_rate['date'] = pd.to_datetime(call_rate['date'])

In [270]:
call_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3248 entries, 0 to 3247
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3248 non-null   datetime64[ns]
 1   종가      3248 non-null   float64       
 2   전일대비    3248 non-null   object        
 3   등락율     3248 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 101.6+ KB


In [271]:
import pandas

dt_index = pandas.date_range(start='20050401', end='20180102')
dt_index = dt_index[::-1]
dt_index = pd.DataFrame(dt_index, columns=['date'])
dt_index
dt_index

,date
0,2018-01-02
1,2018-01-01
2,2017-12-31
3,2017-12-30
4,2017-12-29
...,...
4655,2005-04-05
4656,2005-04-04
4657,2005-04-03
4658,2005-04-02


In [272]:
k.index

RangeIndex(start=0, stop=3248, step=1)

In [273]:
dt_index.index

RangeIndex(start=0, stop=4660, step=1)

In [274]:
k = call_rate.copy()

new = pd.merge(dt_index, k,how='left')


# idx = pd.date_range('20050101', '20171231')
# k.index = pd.DatetimeIndex(k.index)

new

,date,종가,전일대비,등락율
0,2018-01-02,1.52,하락0.06,-3.80%
1,2018-01-01,NaN,NaN,NaN
2,2017-12-31,NaN,NaN,NaN
3,2017-12-30,NaN,NaN,NaN
4,2017-12-29,1.58,상승0.03,+ 1.94%
...,...,...,...,...
4655,2005-04-05,NaN,NaN,NaN
4656,2005-04-04,3.24,하락0.01,-0.31%
4657,2005-04-03,NaN,NaN,NaN
4658,2005-04-02,NaN,NaN,NaN


In [275]:
new = new.fillna(method='ffill')
new = new[['date', '종가']]
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4660 entries, 0 to 4659
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4660 non-null   datetime64[ns]
 1   종가      4660 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 109.2 KB


In [276]:
new

,date,종가
0,2018-01-02,1.52
1,2018-01-01,1.52
2,2017-12-31,1.52
3,2017-12-30,1.52
4,2017-12-29,1.58
...,...,...
4655,2005-04-05,3.23
4656,2005-04-04,3.24
4657,2005-04-03,3.24
4658,2005-04-02,3.24


In [277]:
from dateutil.relativedelta import relativedelta

a_month_earlier = new[['date']].copy()
a_month_earlier= a_month_earlier.date.apply(lambda x: x - relativedelta(months =1))
a_month_earlier

0      2017-12-02
1      2017-12-01
2      2017-11-30
3      2017-11-30
4      2017-11-29
          ...    
4655   2005-03-05
4656   2005-03-04
4657   2005-03-03
4658   2005-03-02
4659   2005-03-01
Name: date, Length: 4660, dtype: datetime64[ns]

In [278]:
a_month_earlier = pd.merge(a_month_earlier, new, how='left')

In [279]:
a_month_earlier

,date,종가
0,2017-12-02,1.48
1,2017-12-01,1.48
2,2017-11-30,1.53
3,2017-11-30,1.53
4,2017-11-29,1.26
...,...,...
4655,2005-03-05,NaN
4656,2005-03-04,NaN
4657,2005-03-03,NaN
4658,2005-03-02,NaN


In [280]:
result = pd.concat([new,a_month_earlier], axis=1)
result = result[:4630]
result.tail()

,date,종가,date,종가
4625,2005-05-05,3.27,2005-04-05,3.23
4626,2005-05-04,3.21,2005-04-04,3.24
4627,2005-05-03,3.30,2005-04-03,3.24
4628,2005-05-02,3.29,2005-04-02,3.24
4629,2005-05-01,3.29,2005-04-01,3.25


In [287]:
result.columns = ['date','callrate', '한달전date', '한달전callrate']

In [293]:
result['Labelling'] = result.apply(lambda x:
                                   'same' if x['callrate']== x['한달전callrate']
                                  else 'up' if x['callrate'] > x['한달전callrate'] else 'down', axis=1)

C:\Users\leeso\anaconda\envs\crawl\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [294]:
result

,date,callrate,한달전date,한달전callrate,Labelling
2,2017-12-31,1.52,2017-11-30,1.53,down
3,2017-12-30,1.52,2017-11-30,1.53,down
4,2017-12-29,1.58,2017-11-29,1.26,up
5,2017-12-28,1.55,2017-11-28,1.26,up
6,2017-12-27,1.51,2017-11-27,1.26,up
...,...,...,...,...,...
4625,2005-05-05,3.27,2005-04-05,3.23,up
4626,2005-05-04,3.21,2005-04-04,3.24,down
4627,2005-05-03,3.30,2005-04-03,3.24,up
4628,2005-05-02,3.29,2005-04-02,3.24,up


In [295]:
result.to_csv('call_rate_fillna.csv', index  = False)